# Dataset cleaning
- See [setup](SETUP.md) instructions.

In [1]:
import duckdb

In [2]:
db = db = duckdb.default_connection # use the default in-memory database
songs = db.read_csv('./dataset/song_data.csv')
plays = db.read_csv('./dataset/count_data.csv')

## Songs
- Counting rows and unique song_ids indicates that there are repeated song_ids in the dataset.
- There are multiple titles and releases per song_id, indicating that the same song_id can have different titles and releases.

In [3]:
songs.limit(3).show()

┌────────────────────┬───────────────────┬───────────────────────┬──────────────────┬───────┐
│      song_id       │       title       │        release        │   artist_name    │ year  │
│      varchar       │      varchar      │        varchar        │     varchar      │ int64 │
├────────────────────┼───────────────────┼───────────────────────┼──────────────────┼───────┤
│ SOQMMHC12AB0180CB8 │ Silent Night      │ Monster Ballads X-Mas │ Faster Pussy cat │  2003 │
│ SOVFVAK12A8C1350D9 │ Tanssi vaan       │ Karkuteillä           │ Karkkiautomaatti │  1995 │
│ SOGTUKN12AB017F4F1 │ No One Could Ever │ Butter                │ Hudson Mohawke   │  2006 │
└────────────────────┴───────────────────┴───────────────────────┴──────────────────┴───────┘



### Checking for nulls
- There are 15 nulls in the dataset for the title column.

In [4]:
sql = """
SELECT count(*)-count(title) as null_title,
count(*)-count(release) as null_release,
count(*)-count(artist_name) as null_artist_name,
count(*)-count(year) as null_year
FROM songs 
WHERE title IS NULL OR release IS NULL OR artist_name IS NULL OR year IS NULL
"""
db.query(sql).show()

┌────────────┬──────────────┬──────────────────┬───────────┐
│ null_title │ null_release │ null_artist_name │ null_year │
│   int64    │    int64     │      int64       │   int64   │
├────────────┼──────────────┼──────────────────┼───────────┤
│         15 │            0 │                0 │         0 │
└────────────┴──────────────┴──────────────────┴───────────┘



### Counting of rows and assessing the cardinality of main columns
- **Total Rows:** There are 1,000,000 rows in the dataset.
- **Unique Song IDs:** There are 999,056 unique song IDs, indicating that some song IDs are duplicated.
- **Unique Titles:** The dataset contains 702,428 unique titles. This suggests that there are multiple entries with the same title, which could be different versions of the same song, covers, or simply songs with identical titles by different artists.
- **Unique Releases:** There are 149,289 unique release identifiers
- **Unique Artist Names:** There are 72,665 unique artist names in the dataset.

In [5]:
sql = """
SELECT count(*) cnt, count(DISTINCT song_id) song_id_uniqs,
count(DISTINCT title) title_uniqs,
count(DISTINCT release) release_uniqs,
count(DISTINCT artist_name) artist_name_uniqs
FROM songs
"""
db.query(sql).show()

┌─────────┬───────────────┬─────────────┬───────────────┬───────────────────┐
│   cnt   │ song_id_uniqs │ title_uniqs │ release_uniqs │ artist_name_uniqs │
│  int64  │     int64     │    int64    │     int64     │       int64       │
├─────────┼───────────────┼─────────────┼───────────────┼───────────────────┤
│ 1000000 │        999056 │      702428 │        149289 │             72665 │
└─────────┴───────────────┴─────────────┴───────────────┴───────────────────┘



### Check for repeated titles and releases for the same song_id

In [6]:
sql = """
SELECT MAX(title_per_song_id), SUM(IF(title_per_song_id > 1, 1, 0)) title_per_song_id_gt_1,
MAX(release_per_song_id), SUM(IF(release_per_song_id > 1, 1, 0)) release_per_song_id_gt_1
FROM (
    SELECT song_id, count(DISTINCT title) title_per_song_id, count(DISTINCT release) release_per_song_id 
    FROM songs
    GROUP BY song_id
    HAVING count(DISTINCT title) > 1 OR count(DISTINCT release) > 1    
)
"""
db.query(sql).show()

┌────────────────────────┬────────────────────────┬──────────────────────────┬──────────────────────────┐
│ max(title_per_song_id) │ title_per_song_id_gt_1 │ max(release_per_song_id) │ release_per_song_id_gt_1 │
│         int64          │         int128         │          int64           │          int128          │
├────────────────────────┼────────────────────────┼──────────────────────────┼──────────────────────────┤
│                      3 │                    120 │                        3 │                      381 │
└────────────────────────┴────────────────────────┴──────────────────────────┴──────────────────────────┘



### Create a songs table aggregated by song_id

In [7]:
sql = """
CREATE OR REPLACE TABLE songs_cleaned AS
SELECT CAST(row_number() OVER () as INTEGER) as song_num,
song_id, string_agg(title) title, string_agg(release) as release,
MAX(artist_name) as artist_name, CAST(MAX(year) as INTEGER) as year
FROM songs
GROUP BY song_id
"""
db.execute(sql)

In [8]:
sql = """
SELECT count(*) cnt, count(DISTINCT song_id) song_id_uniqs,
count(DISTINCT title) title_uniqs,
count(DISTINCT release) release_uniqs,
count(DISTINCT artist_name) artist_name_uniqs
FROM songs_cleaned
"""
db.query(sql).show()

┌────────┬───────────────┬─────────────┬───────────────┬───────────────────┐
│  cnt   │ song_id_uniqs │ title_uniqs │ release_uniqs │ artist_name_uniqs │
│ int64  │     int64     │    int64    │     int64     │       int64       │
├────────┼───────────────┼─────────────┼───────────────┼───────────────────┤
│ 999056 │        999056 │      702834 │        149697 │             72656 │
└────────┴───────────────┴─────────────┴───────────────┴───────────────────┘



### Creating a clean Pandas DataFrame

In [9]:
df_song = db.query('SELECT song_num, title, release, artist_name, year FROM songs_cleaned').df()
df_song.to_pickle('./data/song_cleaned.pkl')
df_song.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999056 entries, 0 to 999055
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   song_num     999056 non-null  int32 
 1   title        999041 non-null  object
 2   release      999056 non-null  object
 3   artist_name  999056 non-null  object
 4   year         999056 non-null  int32 
dtypes: int32(2), object(3)
memory usage: 30.5+ MB


## Song Plays
- There is play data for only 10,000 songs.
- The play_count has some extreme values that need to be checked.
- Verified referential integrity between plays and songs using aggregated song data (where song_id is unique).

In [10]:
plays.limit(3).show()

┌─────────┬──────────────────────────────────────────┬────────────────────┬────────────┐
│ column0 │                 user_id                  │      song_id       │ play_count │
│  int64  │                 varchar                  │      varchar       │   int64    │
├─────────┼──────────────────────────────────────────┼────────────────────┼────────────┤
│       0 │ b80344d063b5ccb3212f76538f3d9e43d87dca9e │ SOAKIMP12A8C130995 │          1 │
│       1 │ b80344d063b5ccb3212f76538f3d9e43d87dca9e │ SOBBMDR12A8C13253B │          2 │
│       2 │ b80344d063b5ccb3212f76538f3d9e43d87dca9e │ SOBXHDL12A81C204C0 │          1 │
└─────────┴──────────────────────────────────────────┴────────────────────┴────────────┘



In [11]:
sql = """
SELECT count(*) cnt,
count(DISTINCT user_id) user_id_uniqs,
count(DISTINCT song_id) song_id_uniqs,
min(play_count), max(play_count), avg(play_count)
FROM plays
"""
db.query(sql).show()

┌─────────┬───────────────┬───────────────┬─────────────────┬─────────────────┬─────────────────┐
│   cnt   │ user_id_uniqs │ song_id_uniqs │ min(play_count) │ max(play_count) │ avg(play_count) │
│  int64  │     int64     │     int64     │      int64      │      int64      │     double      │
├─────────┼───────────────┼───────────────┼─────────────────┼─────────────────┼─────────────────┤
│ 2000000 │         76353 │         10000 │               1 │            2213 │       3.0454845 │
└─────────┴───────────────┴───────────────┴─────────────────┴─────────────────┴─────────────────┘



In [12]:
sql = """
SELECT count(*) cnt
FROM songs_cleaned as s
INNER JOIN plays p ON s.song_id = p.song_id
"""
db.query(sql).show()

┌─────────┐
│   cnt   │
│  int64  │
├─────────┤
│ 2000000 │
└─────────┘



In [13]:
sql = """
CREATE OR REPLACE TABLE users AS
SELECT CAST(row_number() OVER () as INTEGER) as user_num, user_id
FROM plays
GROUP BY user_id
"""
db.execute(sql)
db.query('select * from users').limit(3).show()

┌──────────┬──────────────────────────────────────────┐
│ user_num │                 user_id                  │
│  int32   │                 varchar                  │
├──────────┼──────────────────────────────────────────┤
│        1 │ 8a01ec8627f2bf34db0778f980d9428a7437f4c8 │
│        2 │ 80c40c79420a3354a9c261ba271fce5b2bb126bf │
│        3 │ 92e58cff7a703184850a063ec74d8a42749f0410 │
└──────────┴──────────────────────────────────────────┘



In [14]:
sql = """
CREATE OR REPLACE TABLE plays_cleaned AS
SELECT u.user_num, s.song_num, p.play_count
FROM plays p
INNER JOIN songs_cleaned s ON p.song_id = s.song_id
INNER JOIN users u ON p.user_id = u.user_id
"""
db.execute(sql)
db.query('select * from plays_cleaned').limit(3).show()

┌──────────┬──────────┬────────────┐
│ user_num │ song_num │ play_count │
│  int32   │  int32   │   int64    │
├──────────┼──────────┼────────────┤
│    70212 │   322777 │          1 │
│    70212 │   167016 │          1 │
│    70212 │   675933 │          1 │
└──────────┴──────────┴────────────┘



### Creating a clean Pandas DataFrame

In [15]:
df_play = db.query('select * from plays_cleaned').df()
df_play.to_pickle('./data/play_cleaned.pkl')
df_play.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   user_num    int32
 1   song_num    int32
 2   play_count  int64
dtypes: int32(2), int64(1)
memory usage: 30.5 MB


## Cross item - song pairs

In [16]:
sql = """
CREATE OR REPLACE TABLE song_pairs AS
SELECT p1.song_num song_num1, p2.song_num song_num2, 
CAST(sum(p2.play_count) AS INTEGER) plays,
CAST(COUNT(DISTINCT p2.user_num) AS INTEGER) users
FROM plays_cleaned p1
INNER JOIN plays_cleaned p2 ON p1.user_num = p2.user_num
WHERE p1.song_num != p2.song_num
GROUP BY p1.song_num, p2.song_num
"""
db.execute(sql)
db.query("SELECT count(*) cnt FROM song_pairs").show()

┌──────────┐
│   cnt    │
│  int64   │
├──────────┤
│ 42597082 │
└──────────┘



In [17]:
df_song_pairs = db.query("SELECT * cnt FROM song_pairs").df()
df_song_pairs.to_pickle('./data/song_pairs.pkl')
df_song_pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42597082 entries, 0 to 42597081
Data columns (total 4 columns):
 #   Column     Dtype
---  ------     -----
 0   song_num1  int32
 1   song_num2  int32
 2   plays      int32
 3   users      int32
dtypes: int32(4)
memory usage: 650.0 MB


In [18]:
df_song_pairs = None # free memory

### Creating a User (summary) DataFrame

In [19]:
sql = """
SELECT user_num, 
count(DISTINCT s.artist_name) artist_count,
count(DISTINCT s.title) title_count,
count(DISTINCT s.release) release_count,
count(DISTINCT s.year) year_count
FROM plays_cleaned p
INNER JOIN songs_cleaned s ON p.song_num = s.song_num
GROUP BY user_num
"""
df_user = db.query(sql).df()
df_user.to_pickle('./data/user.pkl')
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76353 entries, 0 to 76352
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   user_num       76353 non-null  int32
 1   artist_count   76353 non-null  int64
 2   title_count    76353 non-null  int64
 3   release_count  76353 non-null  int64
 4   year_count     76353 non-null  int64
dtypes: int32(1), int64(4)
memory usage: 2.6 MB


## Create a Song (summary) DataFrame

In [20]:
sql = """
SELECT p.song_num, 
count(DISTINCT p.user_num) user_count,
sum(p.play_count) play_count,
FROM plays_cleaned p
INNER JOIN songs_cleaned s ON p.song_num = s.song_num
GROUP BY p.song_num
"""
df_user = db.query(sql).df()
df_user.to_pickle('./data/song_summary.pkl')
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   song_num    10000 non-null  int32  
 1   user_count  10000 non-null  int64  
 2   play_count  10000 non-null  float64
dtypes: float64(1), int32(1), int64(1)
memory usage: 195.4 KB


In [ ]:
db.close()